## マルコフ連鎖モデルでメロディー自動生成

### 概要

✅ マルコフ連鎖モデルを利用したメロディーのパターン生成.<br>
✅ メロディー作成のデモ.

### install

まず以下のライブラリをインストール.

In [ ]:
!pip install music21 -q
!pip install numpy -q
!pip install ipywidgets -q

さらに, 楽譜作成ソフトのインストールも必要です. 以下のリンクより, MuseScoreのインストール(無料)を行なってください.

[musescore](https://musescore.org/ja)

---

### 1. マルコフ連鎖モデル

マルコフ連鎖モデルは, 複数の状態間を確率的に移動するモデルで, 現在の状態から次の状態への遷移確率が現在の状態から決まるという性質を持つ. <br>
このセクションでは, マルコフ連鎖モデルを導入および実装を行う.

#### 1-1. マルコフ連鎖モデル

複数の状態間を確率的に遷移するモデルに**マルコフ連鎖モデル**がある. マルコフ連鎖モデルでは, 現在の状態によって次の状態が確率的に決まるという条件で定義される. マルコフ連鎖モデルは, 系列データ(sequential data)の生成・分析にしばしば用いられる.

> ${\sf Def}$(マルコフ連鎖モデル)<br>
> $\mathcal{S} := \{1, 2, \ldots, n\}$を状態の集合とする. $\mathcal{S}$上の確率変数$X_t\ (t = 0, 1, 2, \ldots)$が**マルコフ連鎖**であるとは次の条件が成り立つことをいう：<br>
> $$p(X_{t+1} = s_{t+1}|X_t = s_t, X_{t-1} = s_{t-1}, \ldots, X_{t-m} = s_{t-m}) = p(X_{t + 1} = s_{t+1}|X_t = s_t)$$

マルコフ連鎖モデルの中でも, 遷移確率$p(X_{t + 1} = s_{t+1}|X_t = s_t)$が時間$t$に関して変化しないものを**斉時マルコフ連鎖モデル**といい, 行列$(p_{ij}) := p(X_{t + 1} = j|X_t = i)$を**遷移行列**という. <br>
マルコフ連鎖は, 遷移行列と初期状態を与えるとその後の状態が確率的に決まってしまうため, 遷移行列は最も重要なパラメータである.

#### 1-2. 実装

In [2]:
import numpy as np


class MarkovChainModel:
    def __init__(self, states, transitionMatrix):
        """マルコフ連鎖モデルのクラス

        Args:
            states (array): 状態のリスト
            transitionMatrix (Matrix): 遷移行列
        """
        self._states = states
        self._n = len(states)
        self._transitionMatrix = transitionMatrix

    def sample(self, len, start=0):
        """サンプルを生成します
        Args:
            len (int): 長さ
            start (int, optional): 開始状態のindex. (規定値=0).

        Returns:
            list: 指定した長さのサンプルを返します
        """
        history = [start]
        state = start
        for t in range(len-1):
            # 遷移の更新
            state = np.random.choice(self._n, p=self._transitionMatrix[state])
            history.append(state)
        return [self._states[i] for i in history]


# 　モデルの作成
model = MarkovChainModel(['I ', 'am ', 'like ', 'a cat.'],  # 状態の集合
                         np.array([  # 遷移行列
                             [0, 1/2, 1/2, 0],
                             [0, 0, 0, 1],
                             [0, 0, 0, 1],
                             [1, 0, 0, 0]]))
# サンプル生成(長さ=10)
print(''.join(model.sample(10)))

I am a cat.I like a cat.I am a cat.I 


---

### 2. メロディー生成

このセクションでは, マルコフ連鎖モデルを用いたメロディー生成について述べる. また簡単なメロディー作成のデモも行う.

#### 2-1. メロディーを生成する

メロディーを作成する場合, 以下のような手順で作成できる.

1. 使用する音の集合(=スケール)を選ぶ.
2. スケールから複数の音を取り出し, 音列を作る.
3. 上で作成した音列にリズムを付加する.

したがって上記の手順のうち, 1. 指定したスケールから音列を作成する 2. リズムを付加する, の2箇所においてマルコフ連鎖モデルを用いることで, メロディーの自動生成が可能である.

#### 2-2. メロディー作成のデモ

##### 2-2-1. 単一メロディー

In [4]:
import numpy as np
from music21 import scale, note, duration, stream
from ipywidgets import interact


cMajScale = scale.MajorScale('c')  # Cメジャースケールを使用
pitchList = [p for p in cMajScale.getPitches('c4', 'g4')]  # 使用音を指定

# 音高の生成パターンのモデル
pitchModel = MarkovChainModel(pitchList,  # 状態の集合
                              np.array([  # 遷移行列
                                  [0, 1/4, 0, 1/4, 1/2],
                                  [1/4, 0, 0, 1/4, 1/2],
                                  [1/2, 1/4, 0, 1/4, 0],
                                  [1/2, 1/4, 0, 0, 1/4],
                                  [0, 1/3, 1/3, 1/3, 0]]))

# リズムの生成パターンのモデル
rhythmicAtom = [duration.Duration(1), duration.Duration(
    1/2), duration.Duration(1/4)]  # quarter, eighth, sixteenth
rythmModel = MarkovChainModel(rhythmicAtom,
                              np.array([  # 遷移行列
                                  [1/4, 1/2, 1/4],
                                  [1/4, 1/2, 1/4],
                                  [0, 1/2, 1/2]]))


@interact(l=10)
def composeMelody(l):
    patterns = 4  # パターン数
    for i in range(patterns):
        # 音列を生成
        sequence = pitchModel.sample(l)
        # リズムを生成
        rythm = rythmModel.sample(l)

        stm = stream.Stream()

        for p, d in zip(sequence, rythm):
            stm.append(note.Note(p, duration=d))

        stm.show()
        stm.show('midi')


interactive(children=(IntSlider(value=10, description='l', max=30, min=-10), Output()), _dom_classes=('widget-…

##### 2-2-2. 2声の場合(メロディー+ベース)

In [5]:
import numpy as np
from music21 import scale, note, pitch, duration, stream
from ipywidgets import interact


cMajScale = scale.MajorScale('c')  # Cメジャースケールを使用
pitchList = [p for p in cMajScale.getPitches('c4', 'g4')]  # 使用音を指定

# 音高の生成パターンのモデル(メロディー)
mPitchModel = MarkovChainModel(pitchList,  # 状態の集合
                               np.array([  # 遷移行列
                                   [0, 1/4, 0, 1/4, 1/2],
                                   [1/4, 0, 0, 1/4, 1/2],
                                   [1/2, 1/4, 0, 1/4, 0],
                                   [1/2, 1/4, 0, 0, 1/4],
                                   [0, 1/3, 1/3, 1/3, 0]]))

# リズムの生成パターンのモデル(メロディー)
mRhythmicAtom = [duration.Duration(1), duration.Duration(
    1/2), duration.Duration(1/4)]  # quarter, eighth, sixteenth
mRythmModel = MarkovChainModel(rhythmicAtom,
                               np.array([  # 遷移行列
                                   [1/4, 1/2, 1/4],
                                   [1/4, 1/2, 1/4],
                                   [0, 1/2, 1/2]]))

# 音高の生成パターンのモデル(ベース)
bPitchModel = MarkovChainModel([pitch.Pitch(p) for p in ['c3', 'd3', 'g2']],  # 状態の集合
                               np.array([  # 遷移行列
                                   [0, 1/2, 1/2],
                                   [1/2, 0, 1/2],
                                   [1/3, 1/3, 1/3]]))

# リズムの生成パターンのモデル(ベース)
bRhythmicAtom = [duration.Duration(1), duration.Duration(1), duration.Duration(
    1/2), duration.Duration(3/4)]  # quarter, eighth, dot-eighth
bRythmModel = MarkovChainModel(rhythmicAtom,
                               np.array([  # 遷移行列
                                   [0, 1/2, 1/2],
                                   [1/4, 1/2, 1/4],
                                   [1/4, 1/2, 1/4]]))


def compose2Voices(l=8, patterns=4):
    mPatterns = []
    bPatterns = []

    for i in range(patterns):
        # 音列・リズムを生成(メロディー)
        mSequence, mRythm = mPitchModel.sample(l, start=4), mRythmModel.sample(l)
        # 音列・リズムを生成(ベース)
        bSequence, bRythm = bPitchModel.sample(l), bRythmModel.sample(l)

        mPatterns.append(zip(mSequence, mRythm))
        bPatterns.append(zip(bSequence, bRythm))

    return mPatterns, bPatterns


@interact(i=range(4), j=range(4), l=(1, 10))
def demo(i, j, l=4):
    beats = 32
    
    # メロディー・ベースパターンを作成
    mPatterns, bPatterns = compose2Voices(l, 4)

    score = stream.Score(id="mainsheet")
    mstm = stream.Part(id="melody")
    bstm = stream.Part(id="bass")

    melody = mPatterns[i]
    bass = bPatterns[j]
    
    for p, d in melody:
        mstm.append(note.Note(p, duration=d))

    for p, d in bass:
        bstm.append(note.Note(p, duration=d))

    score.insert(0, mstm)
    score.insert(0, bstm)

    score.show();
    score.show('midi');

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3), value=0), Dropdown(description='j', opti…

--- 

### 3. さらなる改善点に向けて

前セクションで行なったデモに関して, いくつか改善点が挙げられる

**単一メロディー生成の改善点**<br>

✅ マルコフ連鎖モデルの遷移行列を手動で指定したが, システマチックに与える方法はあるか. <br>
✅ 高階のマルコフ連鎖：今回使用したモデルは, 現在の状態から次の状態が決まる単純マルコフ連鎖というモデルである. 対して, より過去の状態にも依存する<u>高階マルコフ連鎖</u>というモデルもあり, これを使うことで, 長い順次進行や特徴的な音列パターンの出現頻度が高まり, より音楽的内容が増すと考えられる.

**2声の場合の改善点**<br>

✅ 今回のデモではメロディとベースを独立に生成したが, メロディーとベースをうまく組み合わせる技術として音楽理論における<u>対位法</u>がある. 対位法の考え方をモデルに組み込むことで, より音楽的内容に富んだ音楽生成が可能だと考えられる.<br>